In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import csv

# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--headless')

# Initialize the driver with the options
driver = webdriver.Chrome(options=chrome_options)

# Go to the website
driver.get('https://www.spotrac.com/nhl/rankings/cap-hit/')

# Scroll several times to load more data
for _ in range(20):  # Adjust as needed
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# Parse the loaded page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find('table')

# Define a CSV file to write the extracted data to
with open('players_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Player Name', 'Team', 'Position', 'Salary']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    rows = table.find_all('tr')
    for row in rows[1:]:  # skipping the header row
        tds = row.find_all('td')
        if len(tds) != 4:  # if for some reason the row doesn't have 4 columns, we skip it
            continue
        
        # Extracting player's name
        player_name_element = tds[1].find('h3')
        player_name = player_name_element.find('a').get_text(strip=True) if player_name_element else None

        # Extracting player's team
        team_img_element = tds[1].find('div', class_='rank-position').find('img')
        team = team_img_element['src'].split('/')[-1].replace('.png', '').upper() if team_img_element else None

        # Extracting player's position
        position = tds[2].get_text(strip=True)

        # Extracting player's salary
        salary = tds[3].find('span', class_='info').get_text(strip=True)

        writer.writerow({
            'Player Name': player_name,
            'Team': team,
            'Position': position,
            'Salary': salary
        })
driver.quit()
